<a href="https://colab.research.google.com/github/Kalana-Lakshan/chatbots/blob/main/Neural_Network_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow numpy scikit-learn

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import json
import random


In [3]:
training_data = [
    {"pattern":"hello","response" : "Hi there! How can I help you?" },
    {"pattern": "hi", "response": "Hello! Nice to meet you!"},
    {"pattern": "how are you", "response": "I'm doing well, thank you!"},
    {"pattern": "what is your name", "response": "I'm NeuralBot, a simple AI chatbot!"},
    {"pattern": "bye", "response": "Goodbye! Have a great day!"},
    {"pattern": "thanks", "response": "You're welcome!"},
    {"pattern": "what time is it", "response": "I can't tell time yet, but I'm learning!"},
    {"pattern": "help", "response": "I can answer simple questions. Try asking about my name or how I am!"}
]

patterns = [data["pattern"].lower() for data in training_data]
responses = [data["response"] for data in training_data]

vocab = set()
for pattern in patterns:
  for word in pattern.split():
    vocab.add(word)
vocab =list(vocab)
print(vocab)

['name', 'are', 'hello', 'time', 'is', 'thanks', 'your', 'hi', 'bye', 'help', 'how', 'it', 'you', 'what']


In [4]:
word_to_index = {word:i for i,word in enumerate(vocab)}
print(word_to_index)

{'name': 0, 'are': 1, 'hello': 2, 'time': 3, 'is': 4, 'thanks': 5, 'your': 6, 'hi': 7, 'bye': 8, 'help': 9, 'how': 10, 'it': 11, 'you': 12, 'what': 13}


In [8]:
def pattern_to_vector(pattern):
  vector = np.zeros(len(vocab))
  for word in pattern.split():
    if word in word_to_index:
      vector[word_to_index[word]] = 1
  return vector

X_train = np.array([pattern_to_vector(p) for p in patterns])
y_train = np.arange(len(responses))

print(X_train.shape)
print(X_train)
print(y_train)

(8, 14)
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]]
[0 1 2 3 4 5 6 7]


In [9]:
# Cell 9: Build a simple neural network
model = keras.Sequential([
    keras.layers.Dense(16, input_shape=(len(vocab),), activation='relu'),
    keras.layers.Dense(8, activation='relu'),
    keras.layers.Dense(len(responses), activation='softmax')
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Neural network created! ✅")
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Neural network created! ✅


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │           240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │            72 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 448 (1.75 KB)

 Trainable params: 448 (1.75 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Cell 10: Train the neural network
print("Training the neural network...")
history = model.fit(
    X_train,
    y_train,
    epochs=200,  # Number of training iterations
    verbose=0  # Set to 1 to see training progress
)

print(f"Training complete! Final accuracy: {history.history['accuracy'][-1]:.2%} ✅")

Training the neural network...
Training complete! Final accuracy: 87.50% ✅


In [11]:
# Cell 11: Create prediction function
def neural_chatbot():
    print("=" * 50)
    print("🧠 NEURAL CHATBOT ACTIVATED")
    print("=" * 50)

    while True:
        user_input = input("\nYou: ").lower().strip()

        if user_input in ['quit', 'exit', 'bye']:
            print("\n🧠 NeuralBot: Goodbye! Hope to chat again!")
            break

        # Convert input to vector
        input_vector = pattern_to_vector(user_input)
        input_vector = np.array([input_vector])  # Add batch dimension

        # Get prediction
        predictions = model.predict(input_vector, verbose=0)
        predicted_index = np.argmax(predictions[0])
        confidence = predictions[0][predicted_index]

        # Only respond if confidence is high enough
        if confidence > 0.5:
            response = responses[predicted_index]
            print(f"\n🧠 NeuralBot: {response}")
        else:
            print(f"\n🧠 NeuralBot: I'm not sure what you mean. Confidence: {confidence:.2f}")

# Run the neural chatbot
print("\nReady to test the neural chatbot!")
neural_chatbot()


Ready to test the neural chatbot!
🧠 NEURAL CHATBOT ACTIVATED

You: hello

🧠 NeuralBot: Hi there! How can I help you?

You: do you know anyhting

🧠 NeuralBot: I'm not sure what you mean. Confidence: 0.18

You: can you help me

🧠 NeuralBot: I'm not sure what you mean. Confidence: 0.17

You: exit

🧠 NeuralBot: Goodbye! Hope to chat again!
